# Creation of a bigram model with a k-smoothing of 1 #

In [11]:
import nltk
nltk.download('treebank')
from nltk.corpus import treebank
from collections import defaultdict, Counter
import math
import random

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


### Print file length ###


In [12]:
files=treebank.fileids()
len(files)

199

### Print first sentence ###

In [13]:
treebank.sents(files[0])

[['Pierre', 'Vinken', ',', '61', 'years', 'old', ',', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'Nov.', '29', '.'], ['Mr.', 'Vinken', 'is', 'chairman', 'of', 'Elsevier', 'N.V.', ',', 'the', 'Dutch', 'publishing', 'group', '.']]

### Split into training and test dataset ###

In [14]:
train_files = treebank.fileids()[:170]
test_files = treebank.fileids()[170:]


### Making sure that they are the correct length ###

In [15]:
print(len(train_files))
print(len(test_files))

170
29


### Create a vocab for words >3 ###
 

In [16]:
token_counter = Counter()

for file in train_files:
    for sent in treebank.sents(file):
        token_counter.update([token.lower() for token in sent])
        

unk_token = "<UNK>"
vocab = {token.lower() for token, count in token_counter.items() if count >= 3}


### Creates a list of bigrams with boundary markers from sentences in training files ###

In [17]:
train_bigrams = []
for file in train_files:
    for sent in treebank.sents(file):
        sent = ['<BOS>'] + [token.lower() if token.lower() in vocab else unk_token for token in sent] + ['<EOS>']
        train_bigrams.extend(nltk.bigrams(sent))
       


### Computing smoothed bigram probabilities  ###

In [18]:
k = 1
bigram_counts = defaultdict(Counter)
for bigram in train_bigrams:
    bigram_counts[bigram[0].lower()][bigram[1].lower()] += 1

bigram_smoothed_probs = defaultdict(Counter)
for w1 in bigram_counts:
    total_count = sum(bigram_counts[w1].values()) + k * len(vocab)
    for w2 in bigram_counts[w1]:
        bigram_smoothed_probs[w1][w2] = (bigram_counts[w1][w2] + k) / total_count
        
        

       
test_bigrams = []
test_bigram_count = 0
for file in test_files:
    for sent in treebank.sents(file):
        sent = ['<BOS>'] + [token.lower() if token.lower() in vocab else unk_token for token in sent] + ['<EOS>'] # Replace with UNK also
        test_bigrams.extend(nltk.bigrams(sent))
        test_bigram_count += len(sent)  
        
    


### Evaluating sum of ln prob ###

In [19]:
total_prob_sum=0.0        
ln_prob_sum = 0.0

for bigram in test_bigrams:
    w1, w2 = bigram
    w1_lower, w2_lower = w1.lower(), w2.lower()
    prob = bigram_smoothed_probs[w1_lower][w2_lower] if w2_lower in bigram_smoothed_probs[w1_lower] else (k / (sum(bigram_counts[w1_lower].values()) + k * len(vocab)))
    ln_prob_sum += math.log(prob)
   

### Print perplexity ###

In [20]:
perplexity = math.exp(-1 * (ln_prob_sum / test_bigram_count))
print(perplexity)

308.0660991988238


### Function to generate sentences based on 3 starting word of the model checking start with \<BOS > ####

In [21]:
def generate_sentence(test_bigrams, bigram_model, start_word):
    if ('<BOS>', start_word.lower()) not in [(bigram[0], bigram[1].lower()) for bigram in test_bigrams if bigram[0] == '<BOS>']:
        raise ValueError("The provided start_word should be the second word of a bigram where the first word is '<BOS>' in the bigram model.")
    
    sentence = [ start_word]
    while sentence[-1] != '<eos>':
        next_word_candidates = [word.lower() for word in list(bigram_model[sentence[-1]].keys())]
        next_word_probs = list(bigram_model[sentence[-1]].values())
        
        if not next_word_candidates:
            sentence.append('<EOS>')
            break
        
        next_word = random.choices(next_word_candidates, next_word_probs)[0]
        
        if next_word == '<unk>':
            continue
        
        sentence.append(next_word)
    
    return sentence[:-1] # exclude <EOS>


### Generate the sentences with starting words 'if', 'an, 'for' ###

The sentences do not appear to convey any meaningful information or follow a coherent narrative or theme. Also the pereplexity seems to be higher for the lowercase model which indicates faulty code or small-unusual dataset.

In [22]:
start_words = ['if', 'an', 'the']
generated_sentences = []

for start_word in start_words:
    generated_sentence = generate_sentence(test_bigrams, bigram_smoothed_probs, start_word.lower())
    generated_sentences.append(generated_sentence)
    print(' '.join(generated_sentence))

if and continued *-1 push up in ual 's corporate defendants were a small company said 0 gencorp 's foreign ministry of the magnetic fields plc 's telling the direct competitor enters the quota of $ 5,000 *u* campaign chairman of mr. simmons owns about one-third of stag 's face amount of the executives .
an aggressive and freedom of the issue .
the 1979 -lrb- or sales *ich*-1 mr. madison .
